<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/MachineLearning/9_SeriesDeTiempo/ejercicios/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>

# Series de Tiempo

## Dataset: Flujo Vehicular por Unidades de Peaje AUSA

**Podés descargar el dataset [aquí](https://unket.s3.sa-east-1.amazonaws.com/data/flujo-vehicular-2017_2021_illia.csv). Deberás descargar -  al menos para comenzar - el archivo "flujo-vehicular-2017_2021_illia.csv". Los datasets originales los podés encontrar [aquí](https://data.buenosaires.gob.ar/dataset/flujo-vehicular-por-unidades-de-peaje-ausa). Nosotros ya hicimos un preprocesamiento para que sea más sencillo trabajar con los dataset durante la clase. Si querés ver cómo es ese preprocesamiento, podés mirar el notebook "PreproDatasets.ipynb".**

En este notebook te dejamos unas celdas para que puedas comenzar a trabajar con este dataset. También te dejamos algunas propuestas para que explores. ¡No te preocupes si no llegas a probarlas todas durante la clase!

Las secciones del notebook son las siguientes:

1. Exploración de datos
1. Objetivos del análisis
1. Componentes de la serie
1. Predicción a Futuro
1. Para pensar, investigar y, opcionalmente, implementar

#### Bibliografía recomendada

El análisis que haremos es muy similar al que se encuentra en el [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) de Jake VanderPlas sobre el dataset de pasos de bicicletas en el Puente Fremont de Seattle, EEUU. Recomendamos las secciones "Working with Time Series" e "In Depth: Linear Regression". También, recomendamos chusmear el libro "Interpretable Machine Learning", de Christoph Molnar, en particular la sección "5.1 Linear Regression", donde pueden encontrar otro ejemplo similar.


## 1. Exploración de datos

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿Cuáles son los días y horarios de mayor tráfico? También, puedes estudiar autocorrelaciones, resampleos, etc. ¿La serie posee valores atípicos? Si es así, interpreta, si es necesario con información auxiliar. 

Nosotros te dejamos unas celdas para comenzar.


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

In [ ]:
data = pd.read_csv('https://unket.s3.sa-east-1.amazonaws.com/data/flujo-vehicular-2017_2021_illia.csv', 
                   parse_dates = [1])
data.head()

Combinamos la columna `fecha` y `hora_inicio`,


In [ ]:
data['fecha'] = pd.to_datetime(data.fecha) + pd.to_timedelta(data.hora_inicio, unit = 'h')
data.head()

Para simplificar el análisis, vamos a sumar la `cantidad_pasos` a lo largo de todas las formas de pago y tipos de vehículo. Solamente vamos a separar por `sentido`.

In [ ]:
data_reducida = data.groupby(['fecha', 'sentido']).cantidad_pasos.sum()
data_reducida = data_reducida.reset_index()
data_reducida

Llevamos el DataFrame a una forma un poquito más cómoda, pivoteando la tabla en la columna `sentido`.

In [ ]:
data_reducida = data_reducida.pivot_table(index = 'fecha',columns=['sentido'], values='cantidad_pasos')
data_reducida.head()

In [ ]:
data_reducida.reset_index(inplace = True)
data_reducida.head()

In [ ]:
data_reducida = data_reducida.rename_axis(None, axis=1) # Para sacarle nombre `sentido` al indice
data_reducida.head()

Veamos si hay valores faltantes

In [ ]:
data_reducida.isna().sum()

Y que no falten fechas

In [ ]:
data_reducida.fecha.diff().value_counts()

Bien, hay muy pocos valores y fechas faltantes. En general, está bueno agregar las fechas que falten y, a veces, imputar valores, pero por hoy vamos a obviarlo.

Sumamos una columna `Total`

In [ ]:
data_reducida['Total'] = data_reducida['Centro'] + data_reducida['Provincia']

Y graficamos

In [ ]:
data_reducida.plot(x = 'fecha', y = ['Centro', 'Provincia'], alpha = 0.5, figsize = (18,6))

La visualización está un poco saturada. Sin embargo, se llegan a ver algunas características sobresalientes. ¿Cuáles son?

Veamos más de cerca algunos días individuales. Antes de correr la serie, anticipa lo que esperas ver, en particular en cada *sentido* del tránsito.

In [ ]:
dia = datetime.datetime(2017,3,5,0,0) # un domingo. ¿qué pasa si sumás un día?

data_reducida.plot(x = 'fecha', y = ['Centro', 'Provincia'], figsize = (18,6))
plt.xlim(dia,dia + datetime.timedelta(days = 1))
plt.show()

**Ejercicio 1:** Podemos resamplear para obtener una visualización un poco más amigable. Para ello, utiliza la función `resample` de Pandas para obtener la cantidad de pasos **diarios**. Y grafica.

In [ ]:
diario = COMPLETAR.COMPLETAR.sum()
diario.reset_index(inplace = True)
diario.head()

Grafiquemos.

In [ ]:
diario.plot(COMPLETAR, figsize = (18,6))
plt.show()

**Ejercicio 2:** Utilizando la función `groupby` de Pandas, obtén el tráfico promedio **por hora**. Te puede ser útil el comando `data_reducida.fecha.dt.time`.

In [ ]:
data_por_hora = COMPLETAR.COMPLETAR(COMPLETAR).mean()
data_por_hora.reset_index(inplace = True)
data_por_hora

In [ ]:
data_por_hora.plot(style=[':', '--', '-']);

**Ejercicio 3:** De la misma forma, obtén el tráfico por día. Pista: `data_reducida.fecha.dt.dayofweek`.

In [ ]:
data_por_dia = COMPLETAR
data_por_dia.index = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_por_dia.plot(style=[':', '--', '-']);


**Ejercicio 4:** Combina ambos análisis para obtener el gráfico del tráfico por hora para los días de la semana y los días de fin de semana. Te dejamos algunas pistas.

In [ ]:
fin_de_semana = np.where(data_reducida.fecha.dt.dayofweek < 5, 'Semana', 'Fin de semana')
fin_de_semana

In [ ]:
data_por_hora = data_reducida.groupby([COMPLETAR, COMPLETAR]).mean()
data_por_hora

In [ ]:
ticks_por_hora = 4 * 60 * 60 * np.arange(6)

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

data_por_hora.loc['Semana'].plot(ax=ax[0], title='Semana',
                           xticks=ticks_por_hora, style=[':', '--', '-'])
data_por_hora.loc['Fin de semana'].plot(ax=ax[1], title='Fin de semana',
                           xticks=ticks_por_hora, style=[':', '--', '-']);



¿Se te ocurre algo más que te gustaría explorar?

In [ ]:
# HAZLO AQUI SI ES EL CASO

## 2. Objetivos del análisis

Piensa algunos posibles objetivos de este análisis. ¿Qué partes interesadas pueden haber en el análisis de un dataset de estas características? Piensa también en aplicaciones similares pero de otros ámbitos.

COMPLETAR

## 3. Componentes de la serie

Utilizando Prophet, obtén las componentes de la serie para el Total de pasos. Observa qué ocurre si utilizas los datos con resolución horaria y los datos con resolución diaria. ¿Qué componente se agrega? También, explora a mano distintos argumentos del modelo.

Para resolver esta sección, puedes utilizar celdas de los materiales asincrónicos. No te preocupes por ahora en separar los datos en *train* y *test*.

**Nota:** En Colab la librería se llama *fbprophet*, mientras que localmente *prophet*.

In [ ]:
# from prophet import Prophet
# from prophet.diagnostics import cross_validation, performance_metrics
# from prophet.plot import plot_plotly

In [ ]:
data_a_descomponer = COMPLETAR # data_reducida o diario
data_a_descomponer.rename(COMPLETAR, inplace = True)

In [ ]:
# instanciamos modelo
m = COMPLETAR

# fiteamos el modelo
m.COMPLETAR

In [ ]:
# para graficar, hacemos predict
forecast_train = m.COMPLETAR

In [ ]:
forecast_train.head()

In [ ]:
# componentes del forecast
m.plot_components(forecast_train,uncertainty=True)

### 4. Predicción a Futuro

Utilizando Prophet, obtén un modelo para predecir el tráfico Total para el último trimestre del 2019. Empieza por el tráfico diario, luego si tienes tiempo ve a una resolución mayor (horaria). ¿Contra qué modelos de referencia compararías? Si bien estaría bueno que los implementes, aunque sea coméntalos. 

In [ ]:
data_a_predecir = COMPLETAR # data_reducida o diario
data_a_predecir.rename(COMPLETAR, inplace = True)

In [ ]:
mask_train = data_a_predecir.ds < datetime.datetime(2019,10,1) #conveniente para despues
train = COMPLETAR[COMPLETAR].copy()
train.head(10)

In [ ]:
train.tail(10)

In [ ]:
mask_test = (data_a_predecir.ds >= datetime.datetime(2019,10,1)) & \
            (data_a_predecir.ds < datetime.datetime(2020,1,1))

fechas_a_predecir = COMPLETAR[COMPLETAR].ds.values

In [ ]:
# generamos un dataset futuro para hacer la prediccion
futuro = pd.DataFrame({'ds': fechas_a_predecir, 'y': np.nan})
futuro.head()

In [ ]:
# instanciamos modelo
m = COMPLETAR

# fiteamos el modelo en TRAIN
m.COMPLETAR

# predecimos en futuro
forecast = m.COMPLETAR

# para graficar, hacemos predict tambien en train
forecast_train = m.COMPLETAR

# #unimos los dos DFs para visualizar las dos partes, train-prediccion
forecast_final = pd.concat([forecast_train, forecast])

# #plot componentes del forecast
m.plot_components(forecast_train);

In [ ]:
forecast_final.head()

In [ ]:
m.plot(forecast_final)
plt.show()

Comparemos contra los datos originales, separando en *train* y *test*. Observa atentamente la *forma* de las predicciones. ¿Notas algo?

In [ ]:
plt.figure(figsize = (18,6))
plt.plot(forecast_final.ds, forecast_final.yhat, alpha = 0.75, 
         label = 'Predicho', color = 'blue')
plt.fill_between(forecast_final.ds, forecast_final.yhat_lower, forecast_final.yhat_upper,
                 alpha = 0.1, color = 'blue' )


plt.plot(data_a_predecir[mask_train].ds, data_a_predecir[mask_train].y, alpha = 0.75, label = 'Train')
plt.plot(data_a_predecir[mask_test].ds, data_a_predecir[mask_test].y, alpha = 0.75, label = 'Test')

plt.legend()
plt.xlim(datetime.datetime(2019,8,1),datetime.datetime(2020,1,1) )

Con el diario del lunes sabemos que el modelo no va a andar muy bien en el 2020. De todas formas, predice sobre 2020 y observa las predicciones. ¿Hay algo que un pronosticador podría haber hecho en 2019 para anticiparse?¿Siempre se puede predecir?

In [ ]:
mask_2020 = (COMPLETAR) & \
            (COMPLETAR)

fechas_a_predecir = data_a_predecir[COMPLETAR].ds.values

In [ ]:
# generamos un dataset futuro para hacer la prediccion
futuro = pd.DataFrame({'ds': fechas_a_predecir, 'y': np.nan})
futuro.head()

In [ ]:
# predecimos en futuro
forecast = m.COMPLETAR

In [ ]:
plt.figure(figsize = (18,6))
plt.plot(forecast.ds, forecast.yhat, alpha = 0.75, 
         label = 'Predicho', color = 'blue')
plt.fill_between(forecast.ds, forecast.yhat_lower, forecast.yhat_upper,
                 alpha = 0.1, color = 'blue' )

plt.plot(data_a_predecir[mask_2020].ds, data_a_predecir[mask_2020].y, alpha = 0.75, label = '2020')

plt.legend()
plt.show()

## 5. Para pensar, investigar y, opcionalmente, implementar

¿Cómo incorporarías la información sobre tipo de vehículo, forma de pago, sentido, día de la semana, etc.? En lo que respecta a las predicciones, ¿esperas que mejoren o empeoren?